In [1]:
import numpy as np
import pandas as pd

In [10]:
portfolio_input_file = "../Datasets/portfolio.csv"
profile_input_file = "../Datasets/profile.csv"
transcript_input_file = "../Datasets/transcript.csv"

In [11]:
portfolio_raw = pd.read_csv(portfolio_input_file, index_col=0)
profile_raw = pd.read_csv(profile_input_file, index_col=0)
transcript_raw = pd.read_csv(transcript_input_file, index_col=0)

#### Reading Raw Datasets

In [20]:
portfolio_df = portfolio_raw.copy()
portfolio_df.info()
portfolio_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [21]:
profile_df = profile_raw.copy()
profile_df.info()
profile_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 796.9+ KB


,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [22]:
profile_df["became_member_on"] = pd.to_datetime(profile_df["became_member_on"], format="%Y%m%d")
profile_df.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


In [23]:
transcript_df = transcript_raw.copy()
transcript_df.info()
transcript_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 11.7+ MB


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [24]:
transcript_df["event"].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

#### Offer Events

In [199]:
offer_completed_df = transcript_df.copy()[transcript_df["event"] == "offer completed"].reset_index(drop=True)
# Extract offer id
offer_completed_df["offer_id"] = [eval(value_dict).get("offer_id") for value_dict in offer_completed_df["value"]]
# Drop event and value columns
offer_completed_df = offer_completed_df.drop(["event", "value"], axis=1)
# Rename time column
offer_completed_df = offer_completed_df.rename(columns={"time":"completed_time"})
offer_completed_df.info()
offer_completed_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33579 entries, 0 to 33578
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   person          33579 non-null  object
 1   completed_time  33579 non-null  int64 
 2   offer_id        33579 non-null  object
dtypes: int64(1), object(2)
memory usage: 787.1+ KB


,person,completed_time,offer_id
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5
1,fe97aa22dd3e48c8b143116a8403dd52,0,fafdcd668e3743c1bb461111dcafc2a4
2,629fc02d56414d91bca360decdfa9288,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
3,676506bad68e4161b9bbaffeb039626b,0,ae264e3637204a6fb9bb56bc8210ddfd
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [200]:
offer_received_df = transcript_df.copy()[transcript_df["event"] == "offer received"].reset_index(drop=True)
# Extract offer id
offer_received_df["offer_id"] = [eval(value_dict).get("offer id") for value_dict in offer_received_df["value"]]
# Drop event and value columns
offer_received_df = offer_received_df.drop(["event", "value"], axis=1)
# Rename time column
offer_received_df = offer_received_df.rename(columns={"time":"received_time"})

offer_received_df.info()
offer_received_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   person         76277 non-null  object
 1   received_time  76277 non-null  int64 
 2   offer_id       76277 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


,person,received_time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [201]:
offer_viewed_df = transcript_df.copy()[transcript_df["event"] == "offer viewed"].reset_index(drop=True)
# Extract offer id
offer_viewed_df["offer_id"] = [eval(value_dict).get("offer id") for value_dict in offer_viewed_df["value"]]
# Drop event and value columns
offer_viewed_df = offer_viewed_df.drop(["event", "value"], axis=1)
# Rename time column
offer_viewed_df = offer_viewed_df.rename(columns={"time":"viewed_time"})

offer_viewed_df.info()
offer_viewed_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57725 entries, 0 to 57724
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   person       57725 non-null  object
 1   viewed_time  57725 non-null  int64 
 2   offer_id     57725 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


,person,viewed_time,offer_id
0,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d
1,d1ede868e29245ea91818a903fec04c6,0,5a8bc65990b245e5a138643cd4eb9837
2,102e9454054946fda62242d2e176fdce,0,4d5c57ea9a6940dd891ad53e9dbe8da0
3,02c083884c7d45b39cc68e1314fec56c,0,ae264e3637204a6fb9bb56bc8210ddfd
4,be8a5d1981a2458d90b255ddc7e0d174,0,5a8bc65990b245e5a138643cd4eb9837


##### Match offer viewed to offer received

In [202]:
# Sort by ascending time
offer_received_df = offer_received_df.sort_values(by=["received_time", "person", "offer_id"])
offer_viewed_df = offer_viewed_df.sort_values(by=["viewed_time", "person", "offer_id"])

# Rows are first matched using person and offer id. 
# Then, the first row in offer_viewed_df where viewed_time >= received_time is used to match to entry in offer_received.
# Assumption: Every offer_viewed entry can be tied to only 1 offer_received entry. View time >= Received time.
offers_rec_view_df = pd.merge_asof(offer_received_df, offer_viewed_df, by=["person", "offer_id"], 
                          left_on="received_time", right_on="viewed_time", direction="forward")

In [203]:
# Check for duplicates
# Mark duplicates as True execpt for the last occurence. Retain last occurence. 
offer_viewed_duplicates_index_list = offers_rec_view_df[
    offers_rec_view_df.duplicated(subset=["person", "offer_id", "viewed_time"], keep="last") 
    & offers_rec_view_df["viewed_time"].notna()].sort_values(["person", "received_time"]).index.to_list()

# Change duplicated offer viewed time to NA for the first occurence
offers_rec_view_df.loc[offer_viewed_duplicates_index_list, "viewed_time"] = pd.NA

offers_rec_view_df.info()
offers_rec_view_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   person         76277 non-null  object 
 1   received_time  76277 non-null  int64  
 2   offer_id       76277 non-null  object 
 3   viewed_time    57725 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.3+ MB


,person,received_time,offer_id,viewed_time
0,0011e0d4e6b944f998e987f904e8c1e5,0,3f207df678b143eea3cee63160fa8bed,6.0
1,0020c2b971eb4e9188eac86d93036a77,0,fafdcd668e3743c1bb461111dcafc2a4,12.0
2,003d66b6608740288d6cc97a6903f4f0,0,5a8bc65990b245e5a138643cd4eb9837,36.0
3,00426fe3ffde4c6b9cb9ad6d077a13ea,0,5a8bc65990b245e5a138643cd4eb9837,36.0
4,005500a7188546ff8a767329a2f7c76a,0,ae264e3637204a6fb9bb56bc8210ddfd,60.0


In [204]:
# Check if distribution of offer viewed_time tallies with the original
offers_rec_view_df["viewed_time"].describe() == offer_viewed_df["viewed_time"].describe()

count     True
mean      True
std      False
min       True
25%       True
50%       True
75%       True
max       True
Name: viewed_time, dtype: bool

In [205]:
offers_rec_view_df["viewed_time"].describe()["std"]

199.3176835596394

In [206]:
offer_viewed_df["viewed_time"].describe()["std"]

199.31768355964036

##### Match offer completed to offer received

In [219]:
# Remove duplicates in offer_completed_df
offer_completed_df_cleaned = offer_completed_df.copy().drop_duplicates().reset_index(drop=True)
print("Number of duplicate rows dropped: ", len(offer_completed_df) - len(offer_completed_df_cleaned))

Number of duplicate rows dropped:  397


In [221]:
# Obtain offer duration information from portfolio_df
offer_duration_df = portfolio_df.copy()[["duration", "id"]].rename(columns={"id": "offer_id"})
# Get offer duration in number of hours
# ASSUMPTION: duration is given as number of days
offer_duration_df["duration"] = offer_duration_df["duration"] * 24 

offer_duration_df

,duration,offer_id
0,168,ae264e3637204a6fb9bb56bc8210ddfd
1,120,4d5c57ea9a6940dd891ad53e9dbe8da0
2,96,3f207df678b143eea3cee63160fa8bed
3,168,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,240,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,168,2298d6c36e964ae4a3e7e9706d1fb8c2
6,240,fafdcd668e3743c1bb461111dcafc2a4
7,72,5a8bc65990b245e5a138643cd4eb9837
8,120,f19421c1d4aa40978ebb69ca19b0e20d
9,168,2906b810c7d4411798c6938adc9daaa5


In [222]:
# Merge offer duration information
offer_completed_df_cleaned = offer_completed_df_cleaned.merge(offer_duration_df, on="offer_id")

offer_completed_df_cleaned.info()
offer_completed_df_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33182 entries, 0 to 33181
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   person          33182 non-null  object
 1   completed_time  33182 non-null  int64 
 2   offer_id        33182 non-null  object
 3   duration        33182 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.0+ MB


,person,completed_time,offer_id,duration
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5,168
1,73ffefd41e9a4ca3ab26b2b3697c6eb7,0,2906b810c7d4411798c6938adc9daaa5,168
2,24115a61df25473e84a8a03f3c98de1a,0,2906b810c7d4411798c6938adc9daaa5,168
3,2481f1fcfbcb4b288e5a03af02d95373,0,2906b810c7d4411798c6938adc9daaa5,168
4,a45b69f1c8554ae7af83e74426ca437a,0,2906b810c7d4411798c6938adc9daaa5,168


In [232]:
# Sort by ascending time
offer_completed_df_cleaned = offer_completed_df_cleaned.sort_values(by=["completed_time", "person", "offer_id"])

# Match 1:
# Rows are first matched using person and offer id. 
# Then, the first completed_time where completed_time >= received_time is used to match.
offers_df = pd.merge_asof(offers_rec_view_df, offer_completed_df_cleaned, by=["person", "offer_id"], 
                          left_on="received_time", right_on="completed_time", direction="forward")

# Match 2:
# Rows are first matched using person and offer id. 
# Then, the nearest completed_time to received_time is used to match. 
offers_df = pd.merge_asof(offers_df, offer_completed_df_cleaned[["person", "offer_id", "completed_time"]], by=["person", "offer_id"], 
                          left_on="received_time", right_on="completed_time", direction="nearest", suffixes=("1", "2"))

offers_df.info()
offers_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   person           76277 non-null  object 
 1   received_time    76277 non-null  int64  
 2   offer_id         76277 non-null  object 
 3   viewed_time      57725 non-null  float64
 4   completed_time1  34925 non-null  float64
 5   duration         34925 non-null  float64
 6   completed_time2  35997 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 4.1+ MB


,person,received_time,offer_id,viewed_time,completed_time1,duration,completed_time2
0,0011e0d4e6b944f998e987f904e8c1e5,0,3f207df678b143eea3cee63160fa8bed,6.0,NaN,NaN,NaN
1,0020c2b971eb4e9188eac86d93036a77,0,fafdcd668e3743c1bb461111dcafc2a4,12.0,54.0,240.0,54.0
2,003d66b6608740288d6cc97a6903f4f0,0,5a8bc65990b245e5a138643cd4eb9837,36.0,NaN,NaN,NaN
3,00426fe3ffde4c6b9cb9ad6d077a13ea,0,5a8bc65990b245e5a138643cd4eb9837,36.0,NaN,NaN,NaN
4,005500a7188546ff8a767329a2f7c76a,0,ae264e3637204a6fb9bb56bc8210ddfd,60.0,NaN,NaN,NaN
